In [1]:
import chipwhisperer as cw

In [2]:
bitstream_path = "C:\\Users\\jhaaa\\Desktop\\cw305_research\\vivado\\soc\\soc.runs\\impl_1\\cw305_top.bit"
scope = cw.scope()
target = cw.target(scope, cw.targets.CW305, bsfile=bitstream_path)

(ChipWhisperer Target WARNING|File CW305.py:591) Using default Verilog defines (c:\Users\jhaaa\Desktop\cw305_research\scripts\my_venv\Lib\site-packages\chipwhisperer/hardware/firmware/cw305/cw305_aes_defines.v); if this is not what you want, provide them via the defines_files argument


In [3]:
target.vccint_set(1.0) #set VCC-INT to 1V
target.pll.pll_outfreq_set(50e6, 1)

In [4]:
target.pll.pll_outfreq_get(0)

9600000.0

In [5]:
# Configure the Scope for capture
scope.gain.db = 40        # Adjust based on signal strength
scope.adc.samples = 5000  # Number of points to capture
scope.adc.offset = 0
scope.adc.basic_mode = "rising_edge"
scope.io.trigger_source = "tio4"
scope.clock.adc_src = "extclk_x4"

(ChipWhisperer Other ERROR|File util.py:419) Setting unknown attribute trigger_source in <class 'chipwhisperer.capture.scopes.cwhardware.ChipWhispererExtra.GPIOSettings'>


In [6]:
target.fpga_write(0x0c, [0x01]) # Enable ROs

In [7]:
target.fpga_write(0x0c, [0x00]) # Disable ROs

In [8]:
# 1. Input as continuous hex strings
key_hex = "000102030405060700010203040506ff"
pt_hex  = "11112233445566778899aabbccddeed8"

# 2. Convert to list of integers
# bytes.fromhex() creates a bytes object; list() turns it into [0x00, 0x01, ...]
key = list(bytes.fromhex(key_hex))[::-1]
plaintext = list(bytes.fromhex(pt_hex))[::-1]

In [9]:
# 2. Write the Key to REG_CRYPT_KEY ('h0a)
target.fpga_write(0x0a, key)

# 3. Write the Plaintext to REG_CRYPT_TEXTIN ('h06)
target.fpga_write(0x06, plaintext)

# 4. Read the registers back to see values set
key_sent = target.fpga_read(0x0a, 16)
formatted_hex_key = key_sent.hex(' ')
print(f"Key: {formatted_hex_key}")
plaintext_sent = target.fpga_read(0x06, 16)
formatted_hex_plaintext = plaintext_sent.hex(' ')
print(f"Plaintext: {formatted_hex_plaintext}")

Key: ff 06 05 04 03 02 01 00 07 06 05 04 03 02 01 00
Plaintext: d8 ee dd cc bb aa 99 88 77 66 55 44 33 22 11 11


In [10]:
target.fpga_write(0x0d, [0x00])
# 5. Trigger the encryption by writing to REG_CRYPT_GO ('h05)
target.fpga_write(0x05, [0x01])

In [11]:
# 6. Read the result from REG_CRYPT_CIPHEROUT ('h09)
ciphertext = target.fpga_read(0x09, 16)
formatted_hex = ciphertext.hex(' ')
print(f"Ciphertext: {formatted_hex}")

Ciphertext: 19 49 91 1c c5 14 7e 50 5d 67 e7 66 56 2e f7 6b


In [12]:
target.fpga_write(0x0d, [0x01])

In [15]:
target.fpga_write(0x07, ciphertext)
key_sent = target.fpga_read(0x0a, 16)
formatted_hex_key = key_sent.hex(' ')
print(f"Key: {formatted_hex_key}")
ciphertext_sent = target.fpga_read(0x07, 16)
formatted_hex_ciphertext = ciphertext_sent.hex(' ')
print(f"Ciphertext in: {formatted_hex_ciphertext}")

Key: ff 06 05 04 03 02 01 00 07 06 05 04 03 02 01 00
Ciphertext in: 19 49 91 1c c5 14 7e 50 5d 67 e7 66 56 2e f7 6b


In [16]:
# 5. Trigger the decryption
target.fpga_write(0x05, [0x01])

In [17]:
# Read the decrypt result result from REG_CRYPT_TEXTOUT ('h08)
plaintextout = target.fpga_read(0x08, 16)
formatted_hex = plaintextout.hex(' ')
print(f"Plaintext out: {formatted_hex}")

Plaintext out: 42 fc 57 5a b9 d7 a0 80 6f a4 84 15 0d 09 ea ba
